In [1]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

import torchsummary as summary
from tqdm import tqdm

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os

import gc

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.empty_cache()

In [4]:
gc.collect()

0

In [5]:
# import data from gdrive
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [6]:
# change range of labels, minimum should be zero
df=pd.read_csv('final_labelled_enron.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
df['result'].unique()
df['result']=df['result']-1
df['result'].unique()
df['content']=df['subject']+' '+df['content']
df.drop('Unnamed: 0',axis=1,inplace=True)
df

array([4, 2, 3, 6, 1, 5, 7], dtype=int64)

In [9]:
df['result'].unique()

array([3, 1, 2, 5, 0, 4, 6], dtype=int64)

In [10]:
# additional filtering to balance classes
'''
df_3=df[(df['result']==3) & (df['content'].str.len()<350)]
df=df[~(df['result']==3)]
df=pd.concat([df,df_3])
df
'''

,message_id,subject,content,result
2,<14187877.1075857584924.JavaMail.evans@thyme>,amazoncom password assistance,amazoncom password assistance greeting from am...,1
5,<29043641.1075857584989.JavaMail.evans@thyme>,wassup,wassup hey freako how have you been im just ta...,1
10,<7789871.1075857584856.JavaMail.evans@thyme>,holiday party,holiday party i know it will be close to impos...,2
12,<4081281.1075857631018.JavaMail.evans@thyme>,mother day,mother day thank you for the beautiful flower ...,1
13,<8847335.1075857631040.JavaMail.evans@thyme>,bnp paribas commodity future ng marketwatch fo...,bnp paribas commodity future ng marketwatch fo...,5
...,...,...,...,...
13553,<33080058.1075845335601.JavaMail.evans@thyme>,re western wholesale activity gas power conf c...,re western wholesale activity gas power conf c...,3
13579,<9002886.1075852513161.JavaMail.evans@thyme>,privileged and confidential attorney client co...,privileged and confidential attorney client co...,3
13580,<20906757.1075852522540.JavaMail.evans@thyme>,confidential attorney work client privilege as...,confidential attorney work client privilege as...,3
13590,<30078399.1075852530017.JavaMail.evans@thyme>,re western wholesale activity gas power conf c...,re western wholesale activity gas power conf c...,3


In [11]:
# import BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['result']
        self.text=[tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [13]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df[['message_id','subject','content']], df['result'], test_size=0.25, stratify=df['result'])

In [14]:
X_train['result']=y_train
X_test['result']=y_test
df_train=X_train
df_val=X_test
df_test=0

In [15]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)
'''
df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)
'''
print(len(df_train),len(df_val))

10018 3340


In [17]:
# BERT classifier architecture, with 7 output classes
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 7)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [19]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [20]:
# hyperparameters
EPOCHS = 50
model = BertClassifier()
LR = 0.000001

In [22]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)
df_train

,message_id,subject,content,result
0,<7177876.1075842135472.JavaMail.evans@thyme>,re nanny,re nanny kay thanks carol kay pm carol st cc n...,4
1,<17486530.1075857696878.JavaMail.evans@thyme>,re pebble,re pebble tycholiz,3
2,<27225296.1075857635728.JavaMail.evans@thyme>,re devon,re devon will call you tonight karen arnold on...,1
3,<20259877.1075857724113.JavaMail.evans@thyme>,authorized trader,authorized trader nan,0
4,<7980073.1075841411318.JavaMail.evans@thyme>,class,class class will be postponed until further no...,3
...,...,...,...,...
10013,<15737103.1075852696474.JavaMail.evans@thyme>,trv notification ng propt pl 10122001,trv notification ng propt pl 10122001 the repo...,6
10014,<9054756.1075851974341.JavaMail.evans@thyme>,ken lay mention at nrsc reception last night,ken lay mention at nrsc reception last night l...,0
10015,<9526932.1075849793495.JavaMail.evans@thyme>,confidential folder to safely pas information ...,confidential folder to safely pas information ...,0
10016,<23550823.1075857657037.JavaMail.evans@thyme>,re swap,re swap will do parker drew on am please respo...,3


In [23]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

201

In [24]:
# freeze first 8 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [28]:
# change datatypes of input data
df_train['message_id']=df_train['message_id'].astype(str)
df_train['subject']=df_train['subject'].astype(str)
df_train['content']=df_train['content'].astype(str)

df_val['message_id']=df_val['message_id'].astype(str)
df_val['subject']=df_val['subject'].astype(str)
df_val['content']=df_val['content'].astype(str)

In [29]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    
    # mini batching
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=30)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=30)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device) # to cuda GPU
                mask = train_input['attention_mask'].to(device) # attention mask
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                l1_loss=0
                
                # for L1 regularization
                a=0
                reg_loss = 0
                for param in model.parameters():
                    a=a+1
                    if a >=201-68:
                        reg_loss += torch.norm(param, 1) 
                
                factor = 0.00001 #lambda for L1 regularization
                l1_loss=factor * reg_loss # L1 loss
                
                # model output
                output = model(input_id, mask)
                
                # loss value
                batch_loss = criterion(output, train_label) + l1_loss
                total_loss_train += batch_loss.item() 
                
                # train accuracy 
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                
                # backpropogation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0
            
            # for validation accuracy
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    
                    # validation output
                    output = model(input_id, mask)
                    
                    # validation loss value
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    # validation accuracy
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [10:24<00:00,  1.87s/it]


Epochs: 1 | Train Loss:  0.360                 | Train Accuracy:  0.267                 | Val Loss:  0.059                 | Val Accuracy:  0.328


100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [10:15<00:00,  1.84s/it]


Epochs: 2 | Train Loss:  0.353                 | Train Accuracy:  0.363                 | Val Loss:  0.056                 | Val Accuracy:  0.375


100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [10:15<00:00,  1.84s/it]


...

Epochs: 93 | Train Loss:  0.303                 | Train Accuracy:  0.895                 | Val Loss:  0.048                 | Val Accuracy:  0.633


100%|████████████████████████████████████████████████████████████████████████████████| 334/334 [10:18<00:00,  1.85s/it]


Epochs: 94 | Train Loss:  0.303                 | Train Accuracy:  0.898                 | Val Loss:  0.048                 | Val Accuracy:  0.626


  9%|███████▎                                                                         | 30/334 [00:56<09:36,  1.90s/it]


KeyboardInterrupt: 

In [38]:
# incase test data is used
'''
%%timeit -n 1 -r 1
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)
'''

Test Accuracy:  0.000
292 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# save model file
torch.save(model.state_dict(), 'BERT_model_freeze.pth')